## Load Dataset

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

dataset = pd.read_csv("https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv")
dataset = dataset.drop(columns=['species'])

for idx in range(dataset.shape[0]):
    if idx < 10:
        dataset.loc[idx,'id'] = f'id00{idx}'
    elif idx < 100:
        dataset.loc[idx,'id'] = f'id0{idx}'
    else:
        dataset.loc[idx,'id'] = f'id{idx}'

dataset.insert(0, 'id', dataset.pop('id'))

dataset

,id,sepal_length,sepal_width,petal_length,petal_width
0,id000,5.1,3.5,1.4,0.2
1,id001,4.9,3.0,1.4,0.2
2,id002,4.7,3.2,1.3,0.2
3,id003,4.6,3.1,1.5,0.2
4,id004,5.0,3.6,1.4,0.2
...,...,...,...,...,...
145,id145,6.7,3.0,5.2,2.3
146,id146,6.3,2.5,5.0,1.9
147,id147,6.5,3.0,5.2,2.0
148,id148,6.2,3.4,5.4,2.3


## Create Databases for tracking process

In [2]:
import mysql.connector
from mysql.connector import Error

try:
    conn = mysql.connector.connect(host='localhost', user='root', password='admin123')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute('DROP DATABASE IF EXISTS irisDB')
        cursor.execute("CREATE DATABASE irisDB")
        print("irisDB database is created")
except Error as e:
    print(e)

cursor.execute("SHOW DATABASES")
for database in cursor:
  print(database)

irisDB database is created
('information_schema',)
('irisdb',)
('mlflowdb',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


## Create table to store iris data & classification result 

In [4]:
try:
    conn = mysql.connector.connect(host='localhost', database='irisDB', user='root', password='admin123')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database()")
        record = cursor.fetchone()

        cursor.execute('DROP TABLE IF EXISTS clf')
        cursor.execute("CREATE TABLE clf ( \
          executed_at DATETIME NOT NULL, \
          id VARCHAR(255) NOT NULL, \
          class INT NOT NULL)")

        cursor.execute('DROP TABLE IF EXISTS iris;')
        cursor.execute("CREATE TABLE iris ( \
          id VARCHAR(255) NOT NULL, \
          sepal_length FLOAT(2,1) NOT NULL, \
          sepal_width FLOAT(2,1) NOT NULL, \
          petal_length FLOAT(2,1) NOT NULL, \
          petal_width FLOAT(2,1) NOT NULL)")

        for i, row in dataset.iterrows():
            cursor.execute("INSERT INTO irisDB.iris VALUES (%s,%s,%s,%s,%s)", tuple(row))
            conn.commit()
        
        print("Classification result table is created")
        print("Iris table is created")

except Error as e:
    print("Error while connecting to MySQL", e)

print("List of table in database :")
cursor.execute("Show tables;") 
for table in cursor.fetchall():
    print(table)

Classification result table is created
Iris table is created
List of table in database :
('clf',)
('iris',)


## Do classification process according to schedule

In [5]:
! python schedule_task.py

^C


In [6]:
# Check the result (example: for scheduling every 10 minutes)
mydb = mysql.connector.connect(host='localhost', database='irisDB', user='root', password='admin123')
print(pd.read_sql('SELECT * FROM irisDB.clf', con=mydb))

          executed_at     id  class
0 2022-09-15 13:57:13  id126      2
1 2022-09-15 13:57:13  id014      0
2 2022-09-15 13:57:13  id127      2
3 2022-09-15 14:07:17  id149      2
4 2022-09-15 14:07:17  id020      0
5 2022-09-15 14:07:17  id103      2
6 2022-09-15 14:17:21  id004      2
7 2022-09-15 14:17:21  id033      2
8 2022-09-15 14:17:21  id039      2
